In [1]:
import pandas as pd
import numpy as np

A brute-force approach will definitely not work here. But we can use a combinatoric approach, being sure not to double count numbers that are both increasing and decreasing.

We know that every number $1$ digit long is both increasing and decreasing. We can count inductively.

Given we know all increasing numbers with $n-1$ digits long, suppose we want all increasing numbers which are $n$ digits long. We can construct the $n$ digit number form an $n-1$ digit number by adding another digit at the end. Let's say we take an increasing number $k$ that is $n-1$ digits long with maximum digit $k_{\max}$. Note $k_{\max} \geq 1$, assuming no leading $0$s. In order to keep it increasing, we can only append a bigger number than $k_{\max}$ at the end. This gives us $10 - k_{\max}$ choices. The method is similar for decreasing numbers.

In [11]:
dec_cache = {1: {i: 1 for i in range(0,10)}}
dec_cache[1][0] = 0
def decreasing_numbers(n):
    '''
    Counts decreasing numbers n digits long.
    '''

    if n not in dec_cache:
        if n-1 not in dec_cache:
            decreasing_numbers(n-1)

        prev_dict = dec_cache[n-1]
        kmin_dict = {i: 0 for i in range(0,10)}

        for prev_kmin, nums in prev_dict.items():
            for i in range(0, prev_kmin + 1):
                kmin_dict[i] += nums
            
        dec_cache[n] = kmin_dict.copy()
    
    return dec_cache[n]

In [2]:
inc_cache = {1: {i: 1 for i in range(1,10)}}
def increasing_numbers(n):
    '''
    Counts increasing numbers n digits long.
    '''

    if n not in inc_cache:
        if n-1 not in inc_cache:
            increasing_numbers(n-1)

        prev_dict = inc_cache[n-1]
        kmax_dict = {i: 0 for i in range(1,10)}

        for prev_kmax, nums in prev_dict.items():
            for i in range(prev_kmax, 10):
                kmax_dict[i] += nums
            
        inc_cache[n] = kmax_dict.copy()
    
    return inc_cache[n]

In [58]:
incs = 0
decs = 0
inc_decs = 0

for n in range(1,101):
    incs += sum(increasing_numbers(n).values())
    decs += sum(decreasing_numbers(n).values())
    # for each n, the 9 numbers where all digits are the same are double counted
    inc_decs += 9

print(incs + decs - inc_decs)
print('incs:', incs)
print('decs:', decs)
print('inc_decs:', inc_decs)

51161058134250
incs: 4263421511270
decs: 46897636623880
inc_decs: 900


After getting the answer, I googled other solutions and came upon a stars and bars approach (also known as compositions of integers), which also uses combinatorics but is even more nice and elegant. 

Take the problem of generating increasing numbers $< 10^{100}$. As I mentioned in my solution above, we cast the problem instead in terms of digits--in particular, we want to look at all $1$ digit up to $100$ digit numbers. Thus we recast the problem more explicitly, in this solution, to finding all distinct (in increasing order) $10$-tuples such that
$$
a_{\text{lead 0}} + a_1 + a_2 + a_3 + \dots + a_9 = 100
$$ 
where $a_i \geq 0$ represents the number of digit $i$ in the final number. We include $a_{\text{lead 0}}$ to account for numbers with fewer than $100$ digits (and note that there is no $a_0$ since $0$ can only be leading for an increasing number). This is already solved by the stars and bars method to be ${100 + 10 - 1 \choose 10 - 1} = {109 \choose 9}$. However, this includes the solution with all leading $0$s, so $1$ needs to be subtracted out.

Similarly for decreasing numbers, we generate all distinct (in decreasing order) $11$-tuples such that
$$
a_{\text{lead 0}} + a_9 + a_8 + a_7 + \dots + a_0 = 100
$$
And, by stars and bars, we have ${100 + 11 - 1 \choose 11 - 1} = {110 \choose 10}$. As above there are solutions with only $0$. However, there are more to think of here since there are both leading and ending $0$ (as opposed to only leading $0$ above). There are $101$ solutions where $a_{\text{lead 0}} + a_0 = 100$, so we subtract $101$ at the end.

Finally, numbers that are the same digit $k$ times are both increasing and decreasing and so are double counted. There are $900$ of these ($9$ digits and up to $100$ of each digit).

In [54]:
# stars and bars approach
fact = [1]
for i in range(1,200):
    fact.append(fact[-1]*i)

print((fact[109] // (fact[100]*fact[9])) + (fact[110] // (fact[100]*fact[10])) - (1 + 101 + 900))
print('incs:', fact[109] // (fact[100]*fact[9]))
print('decs:', fact[110] // (fact[100]*fact[10]))
print('extras_to_subtract:', 1 + 101 + 900)

51161058134250
incs: 4263421511271
decs: 46897636623981
extras_to_subtract: 1002
